In [1]:
import subprocess
import numpy as np
import pandas as pd

In [14]:
starting_position = ('0'*36, '0'*36)
color_dict = ['BLACK', 'WHITE']

def namer(num):
    s = str(num)
    if len(s) < 2:
        s = '0' + s
    return 'comp0' + s

def wdl(gl):
    if gl == 36:
        return '1/2-1/2'
    elif gl%2 == 0:
        return '0-1'
    else:
        return '1-0'

def run_tournament(i):
    for a in range(30):
        for b in range(30):
            players = (a, b)
            status = 'playing'
            position = starting_position
            results_ = []
            while status == 'playing':
                for c in range(2):
                    command = ['tournamentbot.exe', 
                               str(players[c]), position[0], position[1], color_dict[c], 
                               str(30*(a) + b + 900*len(results)+i*36*900)]
                    result = [i.decode('utf-8') for i in subprocess.check_output(command).splitlines()]
                    position = (result[1], result[2])
                    status = result[3]
                    results_.append(position)
                    if status != 'playing':
                        break
            results.append(results_) 

In [55]:
for i in range(10):
    results = []
    run_tournament(i)

    df = pd.DataFrame(index=np.arange(len(results)), columns=['White', 'Black', 'gamelen'])
    df.loc[:, 'gamelen'] = [len(i) for i in results]
    df.loc[:, 'White'] = df.index.values // 30
    df.loc[:, 'Black'] = df.index.values % 30

    df.loc[:, 'Result'] = df.gamelen.map(wdl)
    df.loc[:, 'White'] = df.White.map(namer)
    df.loc[:, 'Black'] = df.Black.map(namer)

    with open('./results' + str(i) + '.pgn', 'w') as f:
        for row in df.index.values:
            row = df.loc[row, :]
            for cell in ['White', 'Black', 'Result']:
                f.write('[' + cell + ' "' + row.loc[cell] + '"]\n')
            f.write(row.loc['Result'])
            f.write('\n\n')

# Real data

In [2]:
import scipy.signal as sig
rec = lambda x: np.array(list(x)).astype(int).reshape(2,4,9)
def win_detector(p):
        """ Takes in a 2x4x9 position; returns a 1 if it finds a 1
        Used to detect terminal nodes """
        rec = lambda x: np.array(list(x)).astype(int).reshape(2,4,9)
        position = rec(p)
        h_filter = np.array([[[1, 1, 1, 1]]])
        v_filter = np.array([np.array([[1, 1, 1, 1]]).T])
        df_filter = np.eye(4)[np.newaxis, :, :]
        db_filter = np.fliplr(np.eye(4))[np.newaxis, :, :]
        filters = [h_filter, v_filter, df_filter, db_filter]
        iswin = False
        for f in filters:
            res = sig.convolve(position, f, mode='valid')
            iswin = np.any(res==4)
            if iswin:
                break

        return int(iswin)

In [8]:
data = pd.read_csv('../../../Data/3_lrn/Clean/_summaries/all_games_model_input.csv', 
                   names=['player', 'color', 'bp', 'wp', 'zet', 'rt', 'condition', 'group', 'session'])
data.loc[:, 'before'] = data.loc[:, 'bp'] + data.loc[:, 'wp']
data.loc[:, 'gamelen'] = (data.loc[:, 'before']).map(lambda x: np.array(list(map(int, list(x)))).sum())

for idx in data.index.values:
    z = data.loc[idx, 'zet']
    if data.loc[idx, 'color'] == 1:
        z += 36
    _p = list(data.loc[idx, 'before'])
    _p[z] = '1'
    data.loc[idx, 'after'] = ''.join(_p)
    
data.loc[:, 'win'] = data.loc[:, 'after'].map(win_detector)    
data.loc[:, 'endmove'] = (data.gamelen.diff(-1) > 0).map(int)
data.loc[data.color==1, 'White'] = data.loc[data.color==1, 'player']
data.loc[data.color==0, 'Black'] = data.loc[data.color==0, 'player']
data.loc[:, 'Black'].fillna(method='ffill', inplace=True)
data.loc[:, 'White'].fillna(method='ffill', inplace=True)
ends = data.loc[data.endmove==1, :]

In [9]:
ends

,player,color,bp,wp,zet,rt,condition,group,session,before,gamelen,after,win,endmove,White,Black
12,0,0,001000100001010000000000100001000000,000111000000001000000000000000000110,20,1515,2,4,1,0010001000010100000000001000010000000001110000...,12,0010001000010100000010001000010000000001110000...,1,1,1015.0,0.0
29,1010,0,000110000000111000000101100000000000,001001000001000100000010000000101010,33,0,2,4,1,0001100000001110000001011000000000000010010000...,16,0001100000001110000001011000000001000010010000...,1,1,0.0,1010.0
50,0,0,000100000100100100001110100000001100,000011100001011010000001000000100010,19,1589,2,4,1,0001000001001001000011101000000011000000111000...,20,0001000001001001000111101000000011000000111000...,1,1,1015.0,0.0
69,1013,0,000010000000011000001011000010101000,010100100010100000000100100001010000,5,0,2,4,1,0000100000000110000010110000101010000101001000...,18,0000110000000110000010110000101010000101001000...,1,1,0.0,1013.0
88,0,0,000100000000111001000100100001001000,000001000100000010000011010010110000,15,1057,2,4,1,0001000000001110010001001000010010000000010001...,18,0001000000001111010001001000010010000000010001...,1,1,1016.0,0.0
99,1014,0,000000000000011100000001000000010000,000000010100100000000000000000101000,16,0,2,4,1,0000000000000111000000010000000100000000000101...,10,0000000000000111100000010000000100000000000101...,1,1,0.0,1014.0
115,1019,1,000100010000000001100100100000011000,000011100000001100000011000000000000,30,0,2,4,1,0001000100000000011001001000000110000000111000...,15,0001000100000000011001001000000110000000111000...,1,1,1019.0,0.0
132,1020,0,000011000000011000000111000000000010,000100100000100000001000100001010100,32,0,2,4,1,0000110000000110000001110000000000100001001000...,16,0000110000000110000001110000000010100001001000...,1,1,0.0,1020.0
142,1025,1,000100000000001000001000000000110000,000000000000000000000111000000001000,24,0,2,4,1,0001000000000010000010000000001100000000000000...,9,0001000000000010000010000000001100000000000000...,1,1,1025.0,0.0
178,0,1,100011010101000100011101011101101010,011100101010111010100010100010010101,17,1448,2,4,1,1000110101010001000111010111011010100111001010...,35,1000110101010001000111010111011010100111001010...,0,1,0.0,1026.0


In [20]:
def namer(s):
    if s > 999:
        s -= 1000
        s = 'comp' + str(s)
    else:
        s = 'subj' + str(s)
    return s

def wdl(gl):
    if gl == 36:
        return '1/2-1/2'
    elif gl%2 == 0:
        return '0-1'
    else:
        return '1-0'

In [21]:
with open('./results_lrn_real.pgn', 'w') as f:
    for row in ends.index.values:
        f.write('[Black "' + namer(ends.loc[row, 'Black']) + '"]\n')
        f.write('[White "' + namer(ends.loc[row, 'White']) + '"]\n')
        f.write(wdl(ends.loc[row, 'gamelen']))
        f.write('\n\n')